In [ ]:
import cv2
import numpy as np
from numpy.core.defchararray import join, mod
import pandas as pd
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import os
import torch
torch.cuda.empty_cache()
from torch._C import device
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection import fasterrcnn_resnet50_fpn, fasterrcnn_mobilenet_v3_large_fpn,fasterrcnn_resnet50_fpn_v2
from torchvision.models.detection.rpn import AnchorGenerator
from PIL import Image
from torchvision import transforms as torchtrans
from sklearn.model_selection import train_test_split
import matplotlib.patches as patches

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
##Hyper Parameters##
batch_size=2
num_classes=2
num_epoch = 10
lr = 0.01
momentum = 0.9

#**Data Preprocessing** **bold text**

Upload dataset and change the train paths

In [ ]:
TRAIN_PATH = '/content/dataset/train/images'
TRAIN_LABELS_PATH = '/content/dataset/train/rcnn_labels'
VAL_PATH = '/content/dataset/valid/images'
VAL_LABEL_PATH = '/content/dataset/valid/labels'


In [ ]:
class Images(Dataset):
    def __init__(self,imgs_path,labels_path):

        self.imgs_path = imgs_path
        self.labels_path = labels_path
        self.img_name = [img for img in sorted(os.listdir(self.imgs_path))]
        self.label_name = [label for label in sorted(os.listdir(self.labels_path))]

    def __getitem__(self,idx):

        image_path = os.path.join(self.imgs_path,str(self.img_name[idx]))
        img = cv2.imread(image_path)

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img_res = img_rgb/255
        img_res = torch.as_tensor(img_res).to(device)
        img_res = img_res.permute(2, 0, 1)

        label_name = self.img_name[idx][:-3] + "txt"
        label_path = os.path.join(self.labels_path,str(label_name))
        with open(label_path, 'r') as label_file:
            l_count = int(label_file.readline())
            box = []
            for i in range(l_count):
                box.append(list(map(int, label_file.readline().split())))

        target={}
        target["boxes"] = torch.as_tensor(box).to(device)
        area = []
        for i in range(len(box)):

            a = (box[i][2] - box[i][0]) * (box[i][3] - box[i][1])
            area.append(a)
        target["area"] = torch.as_tensor(area).to(device)
        labels = []
        for i in range(len(box)):
            labels.append(1)

        target["image_id"] = torch.as_tensor([idx]).to(device)
        target["labels"] = torch.as_tensor(labels, dtype = torch.int64).to(device)


        return img_res,target

    def __len__(self):
        return len(self.img_name)


def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
train_data = Images(TRAIN_PATH, TRAIN_LABELS_PATH)
val_data = Images(VAL_PATH, VAL_LABEL_PATH)
train_loader = DataLoader(train_data, batch_size=batch_size,
                       shuffle=True, num_workers=0, collate_fn=collate_fn)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=collate_fn)

# **Creating the Model**

Select the model to train by uncommenting

In [ ]:
class Model():
  def __init__(self,num_classes,epochs=30,lr=0.01, momentum=0.9):
    # creating faster rcnn model
    # fasterrcnn_resnet50_fpn, fasterrcnn_mobilenet_v3_large_fpn,fasterrcnn_resnet50_fpn_v2
    # Uncomment any line one line

    # model = fasterrcnn_resnet50_fpn_v2(pretrained=True)
    # model = fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
    # model = fasterrcnn_resnet50_fpn(pretrained=True)

    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(
        in_features, num_classes)
    model.to(device)
    self.model = model
    self.train_losses = []
    self.valid_losses = []
    self.epochs = epochs
    self.lr = lr
    self.momentum = momentum

  def train(self,train_data,val_data):
    param = [param for param in self.model.parameters() if param.requires_grad]

    optimizer = torch.optim.SGD(param,lr=self.lr,momentum=self.momentum)
    best_val = 99999
    for epoch in range(self.epochs):
      tot_loss = 0
      self.model.train()
      for img, target in train_data:
          loss_dict = self.model(img, target)
          loss = sum(loss for loss in loss_dict.values())
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          tot_loss += loss
      print("eppch:{},train_loss:{}".format(epoch, tot_loss))

  def save(self,file_path):
    torch.save(self.model, file_path)


# **Training the model**

In [ ]:
model = Model(2,num_epoch,lr,momentum)
model.train(train_loader,val_loader)

In [ ]:
# saving model into colab
VERSION = '1'
MODEL_SAVE_PATH = './WeaponDetection'+ VERSION + ".pt"
model.save(MODEL_SAVE_PATH)